In [1]:
from browser_use import Agent
from dotenv import load_dotenv
load_dotenv()

import asyncio
import nest_asyncio

nest_asyncio.apply()



INFO     [browser_use] BrowserUse logging setup complete with level info
INFO     [root] Anonymized telemetry enabled. See https://docs.browser-use.com/development/telemetry for more information.


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", #gemini-2.5-pro-exp-03-25
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro-exp-03-25", #gemini-2.5-pro-exp-03-25
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [4]:
from browser_use import Agent
import asyncio
from browser_use import Browser
from browser_use import BrowserConfig

# Basic configuration
config = BrowserConfig(
    headless=True,
    disable_security=True
)

browser = Browser(config=config)



async def main():
    agent = Agent(
        task="Compare the price of gpt-4o and DeepSeek-V3",
        llm=llm,
        browser=browser,
        retry_delay=20,
        max_actions_per_step=15
    )
    result = await agent.run()
    return result


In [5]:
sensitive_data = {'first_name': 'magnus', 'last_name': 'Carlson', 'email': 'abc@cb.com', "phone": "1255532", 'available_from': "April 5, 2025", 
                 'desired_salary': "20k $", "resume_path": "/home/xai/projects/ai-playground/browser/tmp.txt", "years_of_experience": "5"}

sensitive_data = {"first_name": "tej", "email": "tej@abc.com", "years_of_experience": "2"}
initial_actions = [
	{'open_tab': {'url': 'http://localhost:8001/static/index.html'}},
]
initial_actions = [
	{'open_tab': {'url': 'https://job-boards.eu.greenhouse.io/groww/jobs/4528265101'}},
]



In [6]:
import os
import sys
from pathlib import Path

from browser_use.agent.views import ActionResult

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("auto_apply.ipynb"))))
import asyncio
import logging


from browser_use import Agent, Controller
from browser_use.browser.browser import Browser, BrowserConfig
from browser_use.browser.context import BrowserContext

logger = logging.getLogger(__name__)

from pydantic import BaseModel

class ApplicationStatus(BaseModel):
    job_title: str
    job_company: str
    status: bool
    reason: str


controller = Controller(output_model=ApplicationStatus)


@controller.action(
	'Upload file to interactive element with file path ',
)
async def upload_file(index: int, path: str, browser: BrowserContext, available_file_paths: list[str]):
	if path not in available_file_paths:
		return ActionResult(error=f'File path {path} is not available')

	if not os.path.exists(path):
		return ActionResult(error=f'File {path} does not exist')

	dom_el = await browser.get_dom_element_by_index(index)

	file_upload_dom_el = dom_el.get_file_upload_element()

	if file_upload_dom_el is None:
		msg = f'No file upload element found at index {index}'
		logger.info(msg)
		return ActionResult(error=msg)

	file_upload_el = await browser.get_locate_element(file_upload_dom_el)

	if file_upload_el is None:
		msg = f'No file upload element found at index {index}'
		logger.info(msg)
		return ActionResult(error=msg)

	try:
		await file_upload_el.set_input_files(path)
		msg = f'Successfully uploaded file to index {index}'
		logger.info(msg)
		return ActionResult(extracted_content=msg, include_in_memory=True)
	except Exception as e:
		msg = f'Failed to upload file to index {index}: {str(e)}'
		logger.info(msg)
		return ActionResult(error=msg)


In [9]:
from browser_use import Agent
import asyncio
from browser_use import Browser
from browser_use import BrowserConfig
from browser_use import Controller, ActionResult
# Initialize the controller
# controller = Controller()
from typing import Optional

@controller.action('Ask human in case you need anything')
def ask_human(question: str) -> str:
    answer = input(f'\n{question}\nInput: ')
    return ActionResult(extracted_content=answer)

from langchain_core.messages import HumanMessage, SystemMessage
from browser_use import Agent, SystemPrompt

class MySystemPrompt(SystemPrompt):
    """
    A SystemPrompt specifically designed for web browsing tasks,
    adding relevant instructions automatically via get_system_message.
    """
    # Define the specific message for this class
    _SPECIFIC_EXTENSION = (
        "IMPORTANT RULE: Cancel the task if you can't find relevant data required to fill the application. But fill the details that you find relevant but do not submit. Check the job requirement and cancel if the job requires more than 3 YOE."
    )

    # No need to override __init__ unless adding new parameters specific to the child
    # The parent __init__ already does what's needed (loads the template).
    # We *could* explicitly call super().__init__ for clarity/future-proofing:
    # def __init__(self, action_description: str, max_actions_per_step: int = 10):
    #     super().__init__(action_description, max_actions_per_step)

    def get_system_message(self):
        """
        Get the system prompt, appending specific task instructions.

        Returns:
            SystemMessage: Formatted system prompt with added instructions.
        """
        # 1. Get the original SystemMessage from the parent class method
        original_message = super().get_system_message()

        # 2. Get the content of the original message
        original_content = original_message.content

        # 3. Append the specific extension for this class
        extended_content = original_content + "\n\n" + self._SPECIFIC_EXTENSION

        # 4. Create and return a *new* SystemMessage with the extended content
        return SystemMessage(content=extended_content)


# class MySystemPrompt(SystemPrompt):
#     def __init__(self) -> str:
#         # Get existing rules from parent class
#         super().__init__()

#         # Add your custom rules
#         new_rules = """
# 9. MOST IMPORTANT RULE:
# - Fill the details using provided fields only do not fill random data, if there is no data cancel the task.

# """

#         # Make sure to use this pattern otherwise the exiting rules will be lost
#         return f'{existing_rules}\n{new_rules}'



agent = Agent(
    task="Try to fill the job application",
    initial_actions=initial_actions,
    controller=controller,
    llm=llm,   
    browser=browser,
    retry_delay=20,
    max_actions_per_step=15,
    sensitive_data=sensitive_data,
    available_file_paths=["/home/xai/projects/ai-playground/browser/tmp.txt"],
    system_prompt_class=MySystemPrompt,
    # message_context="Do not fill random data, if there is no data cancel the task.",
    generate_gif=True

)

    
# Basic configuration
config = BrowserConfig(
    headless=True,
    disable_security=True
)

browser = Browser(config=config)
async def main():
    result = await agent.run()
    return result


In [10]:
res = await main()

INFO     [agent] 🚀 Starting task: Try to fill the job application
INFO     [controller] 🔗  Opened new tab with https://job-boards.eu.greenhouse.io/groww/jobs/4528265101
INFO     [agent] 📍 Step 1
INFO     [agent] 🤷 Eval: Unknown
INFO     [agent] 🧠 Memory: I have opened the job application page for Web Developer - SDE II at Groww. I need to check the job requirements, specifically the years of experience, before proceeding.
INFO     [agent] 🎯 Next goal: Extract job requirements to check Years of Experience (YOE).
INFO     [agent] 🛠️  Action 1/1: {"extract_content":{"goal":"Extract the job requirements, specifically the years of experience required."}}
INFO     [controller] 📄  Extracted from page
: ```json
{
  "years_of_experience_required": "3+ years"
}
```

INFO     [agent] 📍 Step 2
INFO     [agent] 👍 Eval: Success
INFO     [agent] 🧠 Memory: I extracted the job requirements. The job requires 3+ years of experience.
INFO     [agent] 🎯 Next goal: Cancel the application process as the requ

In [11]:
result = res.final_result()


In [12]:
result

'{"job_title": "Web Developer - SDE II", "job_company": "Groww", "status": false, "reason": "The job requires 3+ years of experience, which is more than the maximum allowed 3 YOE."}'

In [14]:
parsed: ApplicationStatus = ApplicationStatus.model_validate_json(result)


In [15]:
parsed

ApplicationStatus(job_title='Web Developer - SDE II', job_company='Groww', status=False, reason='The job requires 3+ years of experience, which is more than the maximum allowed 3 YOE.')

In [ ]:
class MySystemPrompt(SystemPrompt):
    """
    A SystemPrompt specifically designed for web browsing tasks,
    adding relevant instructions automatically.
    """
    # Define the specific message for this class
    _SPECIFIC_EXTENSION = (
        "When browsing websites, focus on extracting the requested information efficiently. "
        "Avoid getting stuck on irrelevant details or complex JavaScript interactions unless necessary."
    )

    def __init__(
        self,
        action_description: str,
        max_actions_per_step: int = 10,
        override_system_message: Optional[str] = None,
        # You can still allow *further* extensions if desired
        extend_system_message: Optional[str] = None,
    ):
        # 1. Prepare the full extension message:
        #    Start with the specific message for this class.
        #    Then, append any *additional* extension provided when creating an instance.
        final_extension = self._SPECIFIC_EXTENSION
        if extend_system_message:
            final_extension += f"\n{extend_system_message}"

        # 2. Call the parent class's __init__ method.
        #    Pass all the standard arguments.
        #    Pass our 'final_extension' as the 'extend_system_message' argument
        #    to the parent initializer. The parent will handle appending it.
        super().__init__(
            action_description=action_description,
            max_actions_per_step=max_actions_per_step,
            override_system_message=override_system_message,
            extend_system_message=final_extension, # Pass the combined/specific extension here
        )

        # No further steps needed here, as super().__init__ already
        # constructed the final prompt and assigned it to self.system_message.
